In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib 
matplotlib.__version__
import pandas as pd
pd.__version__

'2.3.1'

In [28]:
import sklearn
sklearn.__version__

'1.7.1'

In [ ]:
df = pd.read_csv('data/nordics_weather.csv', parse_dates=True)
# df['snow_change'] = df.set_index(['date','country']).snow_depth.unstack().diff().fillna(0).T.stack().values
# df['snow_change_class'] = [np.sign(k) if k!= 0 else 0 for k in df.snow_change]
# df['snow_change_class'] = (df['snow_change_class']+1).astype(int)

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report



In [ ]:

X = df.drop(columns=["snow_change_class"]).iloc[:,2:].drop(columns='snow_change')   # features
y = df["snow_change_class"]                  # categorical target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Pipeline: scaling + SVM classifier
svm_clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", C=1.0, gamma="scale"))

# Fit
svm_clf.fit(X_train, y_train)

# Predict
y_pred = svm_clf.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.40      0.56      0.47       407
           1       0.49      0.27      0.35       309
           2       0.37      0.35      0.36       380

    accuracy                           0.41      1096
   macro avg       0.42      0.39      0.39      1096
weighted avg       0.42      0.41      0.40      1096



In [59]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import classification_report, confusion_matrix


X = df.drop(columns=["snow_depth"]).iloc[:,2:]# features
y = df["snow_depth"]                  # categorical target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Make pipeline: scaling + SVM regression
svm_reg = make_pipeline(StandardScaler(), SVR(kernel="rbf", C=1.0, epsilon=0.1))

# Fit
svm_reg.fit(X_train, y_train)

# Predict
y_pred = svm_reg.predict(X_test)

# Evaluate
from sklearn.metrics import mean_squared_error, r2_score
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))



MSE: 20277.13606438703
R²: 0.33746954209178015


In [48]:
df_Test = pd.DataFrame(dict(truth=y_test, pred=y_pred))
df_Test.reset_index().groupby(['truth','pred']).count().unstack()

index         
pred      0   1    2
truth               
0       229  52  126
1       131  83   95
2       215  33  132